In [1]:
from zhang_algorithm import Zhang

from copy import deepcopy
import numpy as np
import cv2 as cv

from scipy.optimize import least_squares

# data_dir = 'Dataset1'
# img_list = [0, 4, 9, 33]

data_dir = 'Dataset2'
img_list = [2, 7, 8, 9, 15]

zhang = Zhang(data_dir=data_dir)
valid_images, v_lines_all, h_lines_all, pts_all, H_all = zhang.get_lines_pts_H(
                                                            img_list=img_list
                                                            )
omega = zhang.get_omega(H_all)
K = zhang.get_K(omega)
R_t = zhang.get_R_t(K, H_all)
R_all = [r for r, _ in R_t]
t_all = [t for _, t in R_t]
W_all = np.concatenate(zhang.get_W(R_t), axis=0)

pts_reproj_all, mu0, var0, max0 = zhang.get_reproj_error(
    K=K,
    R_all=R_all,
    t_all=t_all,
    world_pts=zhang.actual_pts_aug,
    img_points=pts_all,
)
print(f"before: {mu0, var0, max0}")

a_x = K[0][0]
s = K[0][1]
x_0 = K[0][2]
a_y = K[1][1]
y_0 = K[1][2]
params_0 = np.array([a_x, s, x_0, a_y, y_0])
params_0 = np.concatenate((params_0, W_all, np.concatenate(t_all, axis=0)))
p_star = least_squares(
    zhang.cost_f,
    params_0,
    method="lm",
    kwargs={"pts_all": pts_all, "actual_pts_aug": zhang.actual_pts_aug},
)
params = p_star['x']
K_star = np.array([[params[0], params[1], params[2]], [0, params[3], params[4]], [0, 0, 1]])
R_all_star = zhang.R_from_W(params[5:5+3*len(pts_all)].reshape(-1,3))
t_all_star = params[5+3*len(pts_all):].reshape(-1, 3)

pts_reproj_all_ref, mu1, var1, max1  = zhang.get_reproj_error(
    K=K_star,
    R_all=R_all_star,
    t_all=t_all_star,
    world_pts=zhang.actual_pts_aug,
    img_points=pts_all,
)
print(f"after: {mu1, var1, max1}")

before: (2.595331667913541, 8.652928390001176, 14.017504340360802)
after: (1.1857514883052738, 0.6211401864117173, 4.180285332307832)


In [7]:
from plot_helper import plot_points